Given Time Series problem aims at determining the sales of a combination of shops and their products for the 35th month given the data from certain months from the  past 34 months of those shops.The approach used in this kernel is using non linear methods like XGBRegressor and not traditional time series methods like ARIMA,SARIMA as performing such methods for so many shops and items is not feasible with such sparse data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Reading all csv files provied

In [ ]:
items=pd.read_csv('../input/items.csv')
sample=pd.read_csv('../input/sample_submission.csv')
test=pd.read_csv('../input/test.csv')
train=pd.read_csv('../input/sales_train.csv')
shops=pd.read_csv('../input/shops.csv')
items_cat=pd.read_csv('../input/item_categories.csv')

Replacing all item_cnt_day where values are -ve to 0 

In [ ]:
train.loc[train['item_cnt_day']<0,'item_cnt_day']=0

Dropping all unnecessary fields from train dataset

In [ ]:
train=train.drop(['date','item_price'],axis=1)

In [ ]:
%%html
<div class='tableauPlaceholder' id='viz1539585124140' style='position: relative'><noscript><a href='#'><img alt='Outliers and Total Nulls ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet6&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Time_Series_3&#47;Sheet6' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet6&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1539585124140');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

An outlier is present which has to be removed


REMOVING OUTLIER

In [ ]:
x3=train[train['item_cnt_day']>1400].loc[:,['shop_id','item_id','date_block_num']]
x3

In [ ]:
xx=train[(train['date_block_num']==33) & (train['shop_id']==12) & (train['item_id']==11373)].loc[:,'item_cnt_day'].mean()

Filling Outlier value with mean value of the sales in 33rd month for item 11373 in shop 12

In [ ]:
train[train['item_cnt_day']>1400]=xx

Summing item_count on Date_block

In [ ]:
x=train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day':'sum'})

In [ ]:
%%html
<div class='tableauPlaceholder' id='viz1539584893376' style='position: relative'><noscript><a href='#'><img alt='No. of Records &#47; Date_Block ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Time_Series_3&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1539584893376');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
<div class='tableauPlaceholder' id='viz1539584936345' style='position: relative'><noscript><a href='#'><img alt='Total Items_Sold Per Date_Block ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Time_Series_3&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1539584936345');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>


Some Basic visualization over the Given Series--
The first visual does gives us an idea that the data for all 34 months hasn't been provided for all shops and items

We will be training our dataset over 34rd date_block_num ,so setting target as item_cnt for 34rd date_block_num and predicting the 35th  month

In [ ]:
x=x.reset_index()
z=x[x['date_block_num']==33]

Preparing training and testing dataset
For both training and test purposes, using last 33 data blocks data
hence-->trainign dataset=0-32 target=33
test dataset=1-33 target=34

In [ ]:
x1=x[x['date_block_num']!=33]
x2=x[x['date_block_num']>0]


Pivotting the table so as to get all item count for different months as features

In [ ]:
x1=x1.pivot_table(index=['shop_id','item_id'],columns=['date_block_num'],values=['item_cnt_day'])
x2=x2.pivot_table(index=['shop_id','item_id'],columns=['date_block_num'],values=['item_cnt_day'])

In [ ]:
x1=pd.merge(test,x1,left_on=['shop_id','item_id'],right_on=['shop_id','item_id'],how='left')
x2=pd.merge(test,x2,left_on=['shop_id','item_id'],right_on=['shop_id','item_id'],how='left')


In [ ]:
z=z.drop(['date_block_num'],axis=1)

Trends In Sale during this time of year in the past

In [ ]:
%%html
<div class='tableauPlaceholder' id='viz1539585702541' style='position: relative'><noscript><a href='#'><img alt='Trend 2 Years ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet5&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Time_Series_3&#47;Sheet5' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet5&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1539585702541');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [ ]:
%%html
<div class='tableauPlaceholder' id='viz1539585740116' style='position: relative'><noscript><a href='#'><img alt='Trend 1 year Back ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet4&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Time_Series_3&#47;Sheet4' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Time_Series_3&#47;Sheet4&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1539585740116');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [ ]:
%%html
<div class='tableauPlaceholder' id='viz1539585790306' style='position: relative'><noscript><a href='#'><img alt='Present Trend ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;76&#47;76NX747XY&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;76NX747XY' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;76&#47;76NX747XY&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1539585790306');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

merging training dataset with given list of shops and corresponding items to train only required combinations using LEFT OUTER JOIN 

In [ ]:
z1=pd.merge(test,z,left_on=['shop_id','item_id'],right_on=['shop_id','item_id'],how='left')
len(z1)

Correlation between Sales in different Date_blocks

In [ ]:
%%html
<div class='tableauPlaceholder' id='viz1539585905552' style='position: relative'><noscript><a href='#'><img alt='Correlation between Various sales Months ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;co&#47;corrTS&#47;Sheet1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='corrTS&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;co&#47;corrTS&#47;Sheet1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1539585905552');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

As most values are NaN after joining,coreelation is very high amongst features
Filling NaN with 0

In [ ]:

for c in x1.columns:
    x1[c]=x1[c].fillna(0)
for c in x2.columns:
    x2[c]=x2[c].fillna(0)
    
for c in z1.columns:
    z1[c]=z1[c].fillna(0)


In [ ]:
x1=x1.reset_index().drop(['shop_id','item_id','ID'],axis=1)


In [ ]:

x1=x1.drop(['index'],axis=1)


Converting Training Dataset  and Testing Dataset to numpy array

In [ ]:
x1=np.array(x1)

In [ ]:
z1=np.array(z1['item_cnt_day'])

In [ ]:
q1=x1
x2=x2.reset_index().drop(['shop_id','item_id','ID'],axis=1)



In [ ]:
x2=x2.drop(['index'],axis=1)
x2=np.array(x2)

Generating weights for various features used in dataset
recent sales given hightest weightage followed by 1 year back and lowest given to 2 year back sales

In [ ]:
weights=[1]*9+[1]*12+[8]*6+[8]*6

Splitting Dataset into ratio of 8:2 for training and validation

In [ ]:
from sklearn.model_selection import train_test_split as tts
xtrain,xtest,ztrain,ztest=tts(q1,z1,train_size=0.7)

Using XGBRegressor for prediction purposes and evalaution metrics as Root-Mean-Squared-Error

In [ ]:
from xgboost.sklearn import XGBRegressor as xgbr
xgb=xgbr(n_estimators=5000,learning_rate=0.1,max_depth=7,sample_weight=weights)
xgb.fit(xtrain,ztrain,eval_set=[(xtrain,ztrain),(xtest,ztest)],eval_metric='rmse',early_stopping_rounds=100,verbose=1)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
print(np.sqrt(mse(ztest,xgb.predict(xtest))))

In [ ]:
a=xgb.predict(x2)


Storing result in csv file

In [ ]:
result=pd.read_csv('../input/test.csv')

In [ ]:
result=result.drop(['shop_id','item_id'],axis=1)
result['item_cnt_month']=a
result.set_index('ID')

Marking items_count predited as -ve to 0

In [ ]:
result.loc[result['item_cnt_month']<0,'item_cnt_month']=0
result.to_csv('result.csv')

In [ ]:
result